# {Analyzing the Impact of COVID-19 Vaccination Campaigns on Public Health Outcomes}📝

![Banner](./assets/banner.jpeg)

## Topic
*What problem are you (or your stakeholder) trying to address?*
📝 <!-- Answer Below -->

How informed public health campaigns can affect the overall outcome of a pandemic or similar outbreak when it comes to receiving vaccination and proper care for said ailment. In my instances people can be ill informed about vaccinations or other forms of treatment for illnesses and have an adverse outlook of them when in reality they are often times just ill-equiped to make a proper judgement due to not having equal access to the information at hand to make their own informed decision.

Some questions that I will use to help fuel this inquiry are:

How have COVID-19 vaccination rates and the availability of vaccine doses evolved over time in various regions?

What is the relationship between vaccination rates and the incidence of COVID-19 cases in different demographic groups?

Are there any patterns or disparities in vaccination rates based on socioeconomic factors or geographic locations?

Though not every question above is not the "Project Question" itself, they are things to keep in mind that might add to the overall assessment of the issue at hand and provide insights as to why certain trends might be found.

## Project Question
*What specific question are you seeking to answer with this project?*
*This is not the same as the questions you ask to limit the scope of the project.*
📝 <!-- Answer Below -->

"How did various public health campaigns for the COVID-19 vaccine affect the outcome of public health in various socioeconomic areas?"

## What would an answer look like?
*What is your hypothesized answer to your question?*
📝 <!-- Answer Below -->

"I believe that there will be a reduction in spread and lower number of overall cases of COVID-19 in regions that had stronger public health campaigns that pushed for the vaccine as opposed to those who did not, notably this would be more likely to occur in areas that are of higher socioeconomic standing." 

## Data Sources
*What 3 data sources have you identified for this project?*
*How are you going to relate these datasets?*
📝 <!-- Answer Below -->

The first is api from [covidactnow](https://covidactnow.org/?s=48083556). This has a lot of in depth and useful data available in both json & csv format.

The second source is from the [US Census Database](https://data.census.gov). This information is very useful in that it will provide the different demographics that will be used in the analysis. There are many .csv files that can be used to produce relevant data that can be used.

The final source is from a csv file **covid_states_history.csv**. This file has information dating up until 2021 about various cases, deaths, etc. that will be further used to analyze the problem at hand.

The way I plan on relating the datasets is that I plan on merging the COVID-19 Vaccination data and Case data sets (API & CSV) in order to have common variables such as data, region, etc. to be used to analyze the relationship between COVID-19 cases and vaccination rates. Furthermore, I plan on relation the Socioeconomic Data (CSV) to both the vaccine and case data using different regions as a common key to explore any patterns based on socioeconomic factors. 





## Approach and Analysis
*What is your approach to answering your project question?*
*How will you use the identified data to answer your project question?*
📝 <!-- Start Discussing the project here; you can add as many code cells as you need -->

My approach to this project is that I will first get all of the data from the various sets imported, merged, and cleaned such that I can begin analyzing it. There are a few different ways that I plan to approach these datasets to analyze the problems at hand. I can potentially perform a time-series analysis using line charts that show vaccination rates & does availability by region over a set amount of time. I can use regression analysis to identify any statistically significant relationships between vaccination rates and COVID-19 case occurrence in terms of different demographic groups and visualize that with scatter plots. Finally, I can use heatmaps to visualize the different vaccine rates based on region and socioeconomic factors.  

In [78]:
# Start your code here
from dotenv import load_dotenv

import os
import csv
import json

import pandas as pd
import numpy as np

from urllib.request import urlretrieve
from zipfile import ZipFile
from sqlalchemy import create_engine, text
from pymongo import MongoClient

import requests
from bs4 import BeautifulSoup

load_dotenv(override=True) 
pd.set_option('display.max_rows', 500)

In [77]:
covid_history_df = pd.read_csv('covid_states_history.csv')

print(covid_history_df.head())



         date state    death  deathConfirmed  deathIncrease  deathProbable  \
0  2021-03-07    AK    305.0             NaN              0            NaN   
1  2021-03-07    AL  10148.0          7963.0             -1         2185.0   
2  2021-03-07    AR   5319.0          4308.0             22         1011.0   
3  2021-03-07    AS      0.0             NaN              0            NaN   
4  2021-03-07    AZ  16328.0         14403.0              5         1925.0   

   hospitalized  hospitalizedCumulative  hospitalizedCurrently  \
0        1293.0                  1293.0                   33.0   
1       45976.0                 45976.0                  494.0   
2       14926.0                 14926.0                  335.0   
3           NaN                     NaN                    NaN   
4       57907.0                 57907.0                  963.0   

   hospitalizedIncrease  ...  totalTestResults  totalTestResultsIncrease  \
0                     0  ...         1731628.0            

In [76]:
census_education_df = pd.read_csv('census_education.csv')

print(census_education_df.head())

                                    Label (Grouping)  \
0                      AGE BY EDUCATIONAL ATTAINMENT   
1                          Population 18 to 24 years   
2                     Less than high school graduate   
3          High school graduate (includes equival...   
4                 Some college or associate's degree   

  United States!!Total!!Estimate United States!!Total!!Margin of Error  \
0                            NaN                                   NaN   
1                     31,254,823                               ±43,699   
2                      3,636,420                               ±40,866   
3                     11,051,703                               ±61,246   
4                     12,373,694                               ±66,464   

  United States!!Percent!!Estimate United States!!Percent!!Margin of Error  \
0                              NaN                                     NaN   
1                              (X)                            

In [75]:
census_health_df = pd.read_csv('census_health.csv')

print(census_health_df.head())

                           Label (Grouping) United States!!Total!!Estimate  \
0  Civilian noninstitutionalized population                    328,309,810   
1                                       AGE                            NaN   
2                             Under 6 years                     22,146,508   
3                             6 to 18 years                     54,595,863   
4                            19 to 25 years                     30,279,766   

  United States!!Total!!Margin of Error United States!!Insured!!Estimate  \
0                               ±17,044                      301,941,990   
1                                   NaN                              NaN   
2                               ±40,831                       21,188,997   
3                               ±58,863                       51,621,455   
4                               ±65,829                       26,353,069   

  United States!!Insured!!Margin of Error  \
0                            

In [80]:
census_income_df = pd.read_csv('census_income.csv')

print(census_income_df.head())

                                  Label (Grouping) United States
0                                           Total:   331,449,281
1                          Population of one race:   297,600,338
2                                      White alone   204,277,273
3                  Black or African American alone    41,104,200
4          American Indian and Alaska Native alone     3,727,135


In [79]:
census_population_df = pd.read_csv('census_population.csv')

print(census_population_df.head())

         Label (Grouping) United States!!Households!!Estimate  \
0                   Total                         129,870,928   
1       Less than $10,000                                5.5%   
2      $10,000 to $14,999                                3.7%   
3      $15,000 to $24,999                                6.8%   
4      $25,000 to $34,999                                7.3%   

  United States!!Households!!Margin of Error  \
0                                   ±136,261   
1                                       ±0.1   
2                                       ±0.1   
3                                       ±0.1   
4                                       ±0.1   

  United States!!Families!!Estimate United States!!Families!!Margin of Error  \
0                        83,304,153                                 ±133,863   
1                              3.4%                                     ±0.1   
2                              1.8%                                     ±0.1   


In [81]:
api_key = os.getenv("COVID_ACTNOW_API_KEY")

url = f"https://api.covidactnow.org/v2/states.json?apiKey={api_key}"

response = requests.get(url)

#Request Check
if response.status_code == 200:
    data = response.json()

    #Pandas DataFrame
    covid_data = pd.DataFrame(data)
    
    print(covid_data.head())
else:
    print(f"Request failed with status code: {response.status_code}")


  fips country state county   hsa hsaName  level   lat          locationId  \
0   02      US    AK   None  None    None  state  None  iso1:us#iso2:us-ak   
1   01      US    AL   None  None    None  state  None  iso1:us#iso2:us-al   
2   05      US    AR   None  None    None  state  None  iso1:us#iso2:us-ar   
3   04      US    AZ   None  None    None  state  None  iso1:us#iso2:us-az   
4   06      US    CA   None  None    None  state  None  iso1:us#iso2:us-ca   

   long  population  hsaPopulation  \
0  None      731545            NaN   
1  None     4903185            NaN   
2  None     3017804            NaN   
3  None     7278717            NaN   
4  None    39512223            NaN   

                                             metrics  \
0  {'testPositivityRatio': 0.097, 'testPositivity...   
1  {'testPositivityRatio': None, 'testPositivityR...   
2  {'testPositivityRatio': 0.106, 'testPositivity...   
3  {'testPositivityRatio': 0.302, 'testPositivity...   
4  {'testPositivityRat

## Resources and References
*What resources and references have you used for this project?*
📝 <!-- Answer Below -->

[covidactnow.org](https://covidactnow.org/?s=48083556)

[US Census Database](https://data.census.gov)

[The Covid Tracking Project](https://covidtracking.com/data/download)


In [82]:
# ⚠️ Make sure you run this cell at the end of your notebook before every submission!
!jupyter nbconvert --to python source.ipynb

[NbConvertApp] Converting notebook source.ipynb to python
[NbConvertApp] Writing 6220 bytes to source.py
